How to fetch current physical data for the planets from the HORIZONS system.

This data is then cut-and-pasted into Skyfield.

In [ ]:
%pylab inline

In [ ]:
from telnetlib import Telnet

def fetch_planet_data():
    t = Telnet('horizons.jpl.nasa.gov', 6775)
    for n in [10] + [100*i + 99 for i in range(1, 10)]:
        t.read_until('Horizons>')
        t.write('%s\r\n' % n)
        yield t.read_until('<cr>:')
        t.write('\r\n')
    t.write('q\r\n')
    print t.read_all()

texts = list(fetch_planet_data())

There!

With that simple routine complete,
we have completed the data fetch,
closed the open Telnet connection to the JPL,
and can now process the data in peace without bothering them with a connection dangling open.

In [ ]:
# Here is what the block for the Sun looks like.

print texts[0]

In [ ]:
# And one of the blocks for a planet.

print texts[1]

In [ ]:
def parse(text):
    for line in text.splitlines():
        if line.strip().startswith('Revised'):
            name = line[28:60].strip()
            yield 'name', name
            continue
        halfway = line.find(' ', 40)
        halves = [line] if (halfway == -1) else [line[:halfway], line[halfway:]]
        for half in halves:
            if '\x1b' in half:
                continue
            if '=' in half:
                name, value = half.rsplit('=', 1)
            elif '~' in half:
                name, value = half.rsplit('~', 1)
            else:
                continue
            yield name.strip(), value.strip()

data = [list(parse(text)) for text in texts]

In [ ]:
masses = []

for items in data:
    for k, v in items:
        if k == 'name':
            name = v
            continue
        if 'Mass' not in k:
            continue
        if '10^' not in k:
            continue
        exponent = k.split('^')[1].split()[0]
        mantissa = v.split('+-')[0]
        mass = float(mantissa) * 10.0 ** float(exponent)
        masses.append((name, mass))

print len(masses)
masses

In [ ]:
radii = []

for items in data:
    for k, v in items:
        if k == 'name':
            name = v
            continue
        if 'adius' not in k:
            continue
        if 'mean' not in k.lower() and 'Pluto' not in k and 'Solar' not in k:
            continue
        print k, v
        radius_km = float(v.split()[0].split('+-')[0].split('(')[0])
        if '10^' in v:
            radius_km *= 10 ** int(v.split('10^')[1].split(')')[0])
        radii.append((name, radius_km))

print len(radii)
radii